# 🚀 Node.js Fundamentals Demonstration

## Skills Showcase for Technical Interviews

This notebook demonstrates core Node.js competencies with **instant feedback** and **traditional autocomplete** (no AI assistance).

### Features Demonstrated:
- ✅ ES6+ Modern JavaScript
- ✅ Async/Await Patterns
- ✅ Module Systems (CommonJS & ES Modules)
- ✅ File System Operations
- ✅ Stream Processing
- ✅ Error Handling
- ✅ Performance Optimization

## 1. Modern JavaScript Features

In [ ]:
// Destructuring and Spread Operator
const user = { name: 'John', age: 30, city: 'NYC' };
const { name, ...rest } = user;

console.log('Name:', name);
console.log('Rest:', rest);

// Template Literals
const greeting = `Hello ${name}, you are ${user.age} years old!`;
console.log(greeting);

// Arrow Functions with Implicit Return
const numbers = [1, 2, 3, 4, 5];
const doubled = numbers.map(n => n * 2);
const sum = numbers.reduce((acc, n) => acc + n, 0);

console.log('Doubled:', doubled);
console.log('Sum:', sum);

// Optional Chaining
const nestedData = { user: { profile: { email: 'john@example.com' } } };
console.log('Email:', nestedData?.user?.profile?.email);
console.log('Phone:', nestedData?.user?.profile?.phone ?? 'Not provided');

## 2. Async/Await & Promise Handling

In [ ]:
// Simulated API calls with different delays
const fetchUser = (id: number): Promise<{id: number, name: string}> => {
  return new Promise((resolve) => {
    setTimeout(() => {
      resolve({ id, name: `User${id}` });
    }, Math.random() * 1000);
  });
};

const fetchUserPosts = (userId: number): Promise<string[]> => {
  return new Promise((resolve) => {
    setTimeout(() => {
      resolve([`Post1 by User${userId}`, `Post2 by User${userId}`]);
    }, Math.random() * 800);
  });
};

// Sequential vs Concurrent execution
const demonstrateAsync = async () => {
  console.time('Sequential');
  const user1 = await fetchUser(1);
  const user2 = await fetchUser(2);
  const user3 = await fetchUser(3);
  console.timeEnd('Sequential');
  
  console.time('Concurrent');
  const [concurrentUser1, concurrentUser2, concurrentUser3] = await Promise.all([
    fetchUser(4),
    fetchUser(5),
    fetchUser(6)
  ]);
  console.timeEnd('Concurrent');
  
  // Error handling with async/await
  try {
    const userWithPosts = await fetchUser(1);
    const posts = await fetchUserPosts(userWithPosts.id);
    console.log('User with posts:', { user: userWithPosts, posts });
  } catch (error) {
    console.error('Failed to fetch data:', error);
  }
};

// Execute the demonstration
await demonstrateAsync();

## 3. File System & Stream Operations

In [ ]:
import * as fs from 'fs/promises';
import * as path from 'path';
import { createReadStream, createWriteStream } from 'fs';
import { Transform } from 'stream';
import { pipeline } from 'stream/promises';

// Create demo data
const demoData = {
  users: [
    { id: 1, name: 'Alice', role: 'admin' },
    { id: 2, name: 'Bob', role: 'user' },
    { id: 3, name: 'Charlie', role: 'moderator' }
  ],
  timestamp: new Date().toISOString()
};

// File operations
const testFile = 'demo_data.json';

try {
  // Write file
  await fs.writeFile(testFile, JSON.stringify(demoData, null, 2));
  console.log('✅ File written successfully');
  
  // Read file
  const fileContent = await fs.readFile(testFile, 'utf-8');
  const parsedData = JSON.parse(fileContent);
  console.log('📁 File content:', parsedData);
  
  // File stats
  const stats = await fs.stat(testFile);
  console.log('📊 File size:', stats.size, 'bytes');
  console.log('📅 Modified:', stats.mtime);
  
  // Clean up
  await fs.unlink(testFile);
  console.log('🗑️ File cleaned up');
  
} catch (error) {
  console.error('❌ File operation failed:', error);
}

## 4. Stream Processing & Data Transformation

In [ ]:
import { Readable, Transform, Writable } from 'stream';

// Create a custom readable stream
class NumberGenerator extends Readable {
  private current = 1;
  private max = 10;
  
  _read() {
    if (this.current <= this.max) {
      this.push(this.current.toString() + '\n');
      this.current++;
    } else {
      this.push(null); // End of stream
    }
  }
}

// Create a transform stream to double numbers
class NumberDoubler extends Transform {
  _transform(chunk: Buffer, encoding: string, callback: Function) {
    const number = parseInt(chunk.toString().trim());
    const doubled = number * 2;
    callback(null, `${number} → ${doubled}\n`);
  }
}

// Create a writable stream to collect results
class ResultCollector extends Writable {
  private results: string[] = [];
  
  _write(chunk: Buffer, encoding: string, callback: Function) {
    this.results.push(chunk.toString().trim());
    callback();
  }
  
  getResults() {
    return this.results;
  }
}

// Demonstrate stream pipeline
const demonstrateStreams = async () => {
  const generator = new NumberGenerator();
  const doubler = new NumberDoubler();
  const collector = new ResultCollector();
  
  console.log('🔄 Processing stream...');
  
  await pipeline(generator, doubler, collector);
  
  console.log('📊 Stream processing results:');
  collector.getResults().forEach(result => console.log(result));
};

await demonstrateStreams();

## 5. Error Handling Patterns

In [ ]:
// Custom error classes
class ValidationError extends Error {
  constructor(message: string, public field: string) {
    super(message);
    this.name = 'ValidationError';
  }
}

class NetworkError extends Error {
  constructor(message: string, public statusCode: number) {
    super(message);
    this.name = 'NetworkError';
  }
}

// Simulated functions that might fail
const validateUser = (user: any) => {
  if (!user.email) {
    throw new ValidationError('Email is required', 'email');
  }
  if (user.age < 18) {
    throw new ValidationError('Must be 18 or older', 'age');
  }
};

const simulateNetworkCall = async (shouldFail = false) => {
  if (shouldFail) {
    throw new NetworkError('Connection timeout', 408);
  }
  return { data: 'Success!' };
};

// Error handling strategies
const demonstrateErrorHandling = async () => {
  // 1. Try-catch with specific error types
  try {
    validateUser({ name: 'John', age: 16 }); // Will fail
  } catch (error) {
    if (error instanceof ValidationError) {
      console.log(`❌ Validation failed for ${error.field}: ${error.message}`);
    } else {
      console.log('❌ Unexpected error:', error);
    }
  }
  
  // 2. Graceful degradation with fallbacks
  const safeNetworkCall = async (retries = 3) => {
    for (let attempt = 1; attempt <= retries; attempt++) {
      try {
        const result = await simulateNetworkCall(attempt < 3); // Fail first 2 attempts
        console.log(`✅ Success on attempt ${attempt}:`, result);
        return result;
      } catch (error) {
        console.log(`⚠️ Attempt ${attempt} failed:`, error.message);
        if (attempt === retries) {
          console.log('❌ All retries exhausted');
          throw error;
        }
        await new Promise(resolve => setTimeout(resolve, 100 * attempt)); // Exponential backoff
      }
    }
  };
  
  await safeNetworkCall();
  
  // 3. Result/Error pattern (functional approach)
  type Result<T, E = Error> = { success: true; data: T } | { success: false; error: E };
  
  const safeOperation = async <T>(operation: () => Promise<T>): Promise<Result<T>> => {
    try {
      const data = await operation();
      return { success: true, data };
    } catch (error) {
      return { success: false, error: error as Error };
    }
  };
  
  const result = await safeOperation(() => simulateNetworkCall(false));
  if (result.success) {
    console.log('🎉 Operation succeeded:', result.data);
  } else {
    console.log('💥 Operation failed:', result.error.message);
  }
};

await demonstrateErrorHandling();

## 6. Performance & Memory Optimization

In [ ]:
// Memory usage monitoring
const getMemoryUsage = () => {
  const usage = process.memoryUsage();
  return {
    rss: Math.round(usage.rss / 1024 / 1024 * 100) / 100,
    heapTotal: Math.round(usage.heapTotal / 1024 / 1024 * 100) / 100,
    heapUsed: Math.round(usage.heapUsed / 1024 / 1024 * 100) / 100,
    external: Math.round(usage.external / 1024 / 1024 * 100) / 100
  };
};

console.log('🔍 Initial memory usage:', getMemoryUsage(), 'MB');

// Inefficient vs Efficient data processing
const largeArray = Array.from({ length: 1000000 }, (_, i) => i);

// Inefficient: Creates intermediate arrays
console.time('Inefficient processing');
const inefficient = largeArray
  .map(x => x * 2)
  .filter(x => x % 4 === 0)
  .slice(0, 1000);
console.timeEnd('Inefficient processing');
console.log('🐌 Inefficient result length:', inefficient.length);
console.log('📊 Memory after inefficient:', getMemoryUsage(), 'MB');

// Efficient: Single pass with generator
function* efficientProcessor(array: number[]) {
  let count = 0;
  for (const item of array) {
    if (count >= 1000) break;
    const doubled = item * 2;
    if (doubled % 4 === 0) {
      yield doubled;
      count++;
    }
  }
}

console.time('Efficient processing');
const efficient = Array.from(efficientProcessor(largeArray));
console.timeEnd('Efficient processing');
console.log('🚀 Efficient result length:', efficient.length);
console.log('📊 Memory after efficient:', getMemoryUsage(), 'MB');

// Debouncing for performance
const debounce = <T extends (...args: any[]) => any>(
  func: T,
  delay: number
): ((...args: Parameters<T>) => void) => {
  let timeoutId: NodeJS.Timeout;
  return (...args: Parameters<T>) => {
    clearTimeout(timeoutId);
    timeoutId = setTimeout(() => func(...args), delay);
  };
};

const expensiveOperation = (input: string) => {
  console.log('🔄 Processing:', input);
  // Simulate expensive operation
};

const debouncedOperation = debounce(expensiveOperation, 300);

// Simulate rapid calls - only the last one should execute
debouncedOperation('input1');
debouncedOperation('input2');
debouncedOperation('input3');

// Wait to see the debounced result
await new Promise(resolve => setTimeout(resolve, 500));
console.log('✅ Debouncing demonstration complete');

## �� Summary

This notebook demonstrated:

### ✅ Core Node.js Skills:
- Modern ES6+ JavaScript features
- Advanced async/await patterns
- File system operations
- Stream processing
- Error handling strategies
- Performance optimization

### 💪 Technical Competencies:
- Memory-efficient programming
- Custom error classes
- Stream transformations
- Debouncing patterns
- Generator functions
- Functional programming concepts

### 🎯 Interview Readiness:
All code runs instantly with traditional autocomplete - no AI assistance required!